In [1]:
import nbimporter
import van_Genuchten as vG

Importing Jupyter notebook from van_Genuchten.ipynb


### Bentheimer Sandstoneの物性値  
参考：V.Voronina, L. Pel and K. Kopinga: The influence of osmotic pressure on poulticing treatments for heritage objects, Material and Structures, vol.46, pp221-231, 2013  
$\phi_{max}$：空隙率[-]  
$K_{sat}$：飽和透水係数  
$\lambda^{'}_P$：水蒸気圧勾配に対する気相水分伝導率  
$\rho$；材料の密度[kg/m3]  
$C$：比熱[J/(kg・K)]  
$\rho$：水の密度[kg/m3]  
$r$：水の相変化熱量  

物性値を書く際における注意事項  
・物性情報は必ず(tem:温度、moisture：水分状態)からなる関数する。（変数を必要としない場合でも書くこと）  
・

【理想】    
・それぞれの名称を重複することなく一度定義で完結させれること。

In [2]:
def prop():
    return {
        'Phimax' : 0.23,
        'Ksat'   : 2.0e-7,
        'LAMDP'  : 2.0E-10,
        'row'    : 1479.25,
        'C'      : 750.0,
        'roww'   : 1000.0,
        'r'      : 4.18605E+3,
        # van-Genuchten用情報
        'Alfa'   : 10.0/98.0,
        'n'      : 2.0,
        'm'      : 1.0 - ( 1.0 / 2.0 ),
        'l'      : 0.5,
    }

## Functions

In [3]:
def getCRowByMiu( temp, miu ):
    p = prop()
    return p['row'] * p['C'] + p['row'] * getPhiByMiu( temp, miu ) * p['r']

In [4]:
def getLAMByMiu( temp, miu ):
    return 1.2

In [5]:
def getPhiByMiu( temp, miu ):
    p = prop()
    return vG.getPhi( p['Phimax'], p['Alfa'], miu, p['m'], p['n'] )

In [6]:
def getMiuByPhi( temp, phi ):
    p = prop()
    return vG.getMiu( p['Phimax'], phi, p['Alfa'], p['m'], p['n'] )                                              

In [7]:
def getDwByMiu( temp, miu ):
    p = prop()
    return p['Ksat'] * vG.getKl( p['Alfa'], miu, p['m'], p['n'], p['l'] )

In [8]:
def getDPByMiu( temp, miu ):
    p = prop()
    Phi = getPhiByMiu( temp, miu )
    Sl  = Phi / p['Phimax']
    return p['LAMDP'] * ( 1.0 - Sl * 0.9 )    

In [9]:
def getDPhiByMiu( temp, miu ):
    p = prop()
    return vG.getDPhi( p['Phimax'], p['Alfa'], miu, p['m'], p['n'] )

In [10]:
temp = 293.15
miu  = -10.0

In [11]:
getCRowByMiu( temp, miu )

2106282.314793056

In [12]:
getLAMByMiu( temp, miu )

1.2

In [13]:
getPhiByMiu( temp, miu )

0.16098357394272111

In [14]:
getMiuByPhi( temp, 0.22 )

-2.988199933567902

In [15]:
getDwByMiu( temp, miu )

1.3666027427457417e-08

In [16]:
getDPByMiu( temp, miu )

7.401285517526175e-11

In [17]:
getDPhiByMiu( temp, miu )

0.0082117717783473342